In [1]:
import os
import subprocess
from bids import BIDSLayout
from pathlib import Path
from multiprocessing import Pool

# Paths
BIDS_DIR = "/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS"
DERIVATIVES_DIR = f"{BIDS_DIR}/derivatives/fmriprep/analysis-forcenobbr_minimal"
ANTS_CMD = "antsApplyTransforms"  # Ensure ANTs is installed

# Initialize PyBIDS layout
layout = BIDSLayout(DERIVATIVES_DIR, validate=False)

In [14]:
sub='01'
ses='09'
anat_masks = layout.get(
            subject=sub, session=ses, datatype="anat",
            desc="brain",suffix="mask", extension="nii.gz"
        )
# get only the brain_mask.nii.gz, not getting the ribon_mask
anat_masks

[<BIDSImageFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/anat/sub-01_ses-09_run-01_desc-brain_mask.nii.gz'>]

In [12]:
anat_masks[0].path

'/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/anat/sub-01_ses-09_run-01_desc-brain_mask.nii.gz'

In [13]:
hmc_boldref = layout.get(
            subject=sub, session=ses, datatype="func",
            desc="hmc",suffix="boldref", extension="nii.gz"
        )
hmc_boldref 


[<BIDSImageFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-01_desc-hmc_boldref.nii.gz'>,
 <BIDSImageFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-02_desc-hmc_boldref.nii.gz'>,
 <BIDSImageFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-03_desc-hmc_boldref.nii.gz'>,
 <BIDSImageFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-04_desc-hmc_boldref.nii.gz'>,
 <BIDSImageFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-05_desc-hmc_boldref.nii.gz'>,
 <BIDSImageFile

In [15]:
# Find the corresponding transformation file
xfm_files = layout.get(
    subject=sub, session=ses, datatype="func",
    desc="hmc",suffix="xfm", extension="txt"
)


In [16]:
xfm_files

[<BIDSFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-01_from-orig_to-boldref_mode-image_desc-hmc_xfm.txt'>,
 <BIDSFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-02_from-orig_to-boldref_mode-image_desc-hmc_xfm.txt'>,
 <BIDSFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-03_from-orig_to-boldref_mode-image_desc-hmc_xfm.txt'>,
 <BIDSFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-04_from-orig_to-boldref_mode-image_desc-hmc_xfm.txt'>,
 <BIDSFile filename='/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/fun

In [ ]:



def process_brain_mask(anat_mask, bold_ref, xfm_txt, output_bold_mask):
    """
    Applies ANTs transformation to map a brain mask from T1w space to BOLD space.
    """
    cmd_ants = [
        ANTS_CMD,
        "-d", "3",
        "-i", anat_mask,
        "-r", bold_ref,
        "-t", xfm_txt,
        "-o", output_bold_mask,
        "-n", "NearestNeighbor"
    ]

    print(f"Processing: {anat_mask} -> {output_bold_mask}")
    subprocess.run(cmd_ants, check=True)


# Get all subjects and sessions
subjects = layout.get_subjects()
sessions = layout.get_sessions()

jobs = []

for sub in subjects:
    for ses in sessions:
        # Find all anatomical brain masks (space-T1w and desc-brain_mask)
        anat_masks = layout.get(
            subject=sub, session=ses, datatype="anat",
            desc="brain",suffix="mask", extension="nii.gz"
                )[0]
        anat_mask_path = anat_mask.path

        # Find corresponding functional bold reference images
        hmc_boldrefs = layout.get(
                    subject=sub, session=ses, datatype="func",
                    desc="hmc",suffix="boldref", extension="nii.gz"
                )

        for bold_ref in hmc_boldrefs:
            bold_ref_path = bold_ref.path

            xfm_file = layout.get(
                subject=sub, session=ses, datatype="func",
                task=bold_ref.task, run=bold_ref.run,
                desc="hmc",suffix="xfm", extension="txt"
                    )
            xfm_txt_path=xfm_file[0].path                    
            # Construct the output path
            output_desc_mask = bold_ref_path.replace(
                "desc-hmc_boldref.nii.gz", "desc-brain_mask.nii.gz"
            )
            
            if all(map(os.path.exists, [anat_mask_path, bold_ref_path, xfm_txt_path])):
                jobs.append((anat_mask_path, bold_ref_path, xfm_txt_path, output_desc_mask))
            else:
                print(f"Skipping: Missing files for {sub} {ses}")

# Use multiprocessing for faster processing
if __name__ == "__main__":
    with Pool(processes=8) as pool:  # Adjust number of processes
        pool.starmap(process_brain_mask, jobs)

    print("Processing completed.")


In [17]:
bold_ref=hmc_boldref[0]

In [21]:
            xfm_file = layout.get(
                subject=sub, session=ses, datatype="func",
                task=bold_ref.task, run=bold_ref.run,
                desc="hmc",suffix="xfm", extension="txt"
                    )

/export/home/tlei/tlei/soft/miniconda3/envs/votcloc/lib/python3.11/site-packages/bids/layout/models.py:229: UserWarning: Accessing entities as attributes is deprecated as of 0.7. Please use the .entities dictionary instead (i.e., .entities['task'] instead of .task.
  warnings.warn("Accessing entities as attributes is deprecated as "
/export/home/tlei/tlei/soft/miniconda3/envs/votcloc/lib/python3.11/site-packages/bids/layout/models.py:229: UserWarning: Accessing entities as attributes is deprecated as of 0.7. Please use the .entities dictionary instead (i.e., .entities['run'] instead of .run.
  warnings.warn("Accessing entities as attributes is deprecated as "


In [24]:
xfm_file[0].path

'/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-01_from-orig_to-boldref_mode-image_desc-hmc_xfm.txt'

In [19]:
bold_ref.task

/export/home/tlei/tlei/soft/miniconda3/envs/votcloc/lib/python3.11/site-packages/bids/layout/models.py:229: UserWarning: Accessing entities as attributes is deprecated as of 0.7. Please use the .entities dictionary instead (i.e., .entities['task'] instead of .task.
  warnings.warn("Accessing entities as attributes is deprecated as "


'fLoc'

In [20]:
bold_ref.run

/export/home/tlei/tlei/soft/miniconda3/envs/votcloc/lib/python3.11/site-packages/bids/layout/models.py:229: UserWarning: Accessing entities as attributes is deprecated as of 0.7. Please use the .entities dictionary instead (i.e., .entities['run'] instead of .run.
  warnings.warn("Accessing entities as attributes is deprecated as "


01

In [18]:
bold_ref_path = bold_ref.path
bold_ref_path

'/bcbl/home/public/Gari/VOTCLOC/main_exp/BIDS/derivatives/fmriprep/analysis-forcenobbr_minimal/sub-01/ses-09/func/sub-01_ses-09_task-fLoc_run-01_desc-hmc_boldref.nii.gz'

In [ ]:



def process_brain_mask(anat_mask, bold_ref, xfm_txt, output_bold_mask):
    """
    Applies ANTs transformation to map a brain mask from T1w space to BOLD space.
    """
    cmd_ants = [
        ANTS_CMD,
        "-d", "3",
        "-i", anat_mask,
        "-r", bold_ref,
        "-t", xfm_txt,
        "-o", output_bold_mask,
        "-n", "NearestNeighbor"
    ]

    print(f"Processing: {anat_mask} -> {output_bold_mask}")
    subprocess.run(cmd_ants, check=True)


# Get all subjects and sessions
subjects = layout.get_subjects()
sessions = layout.get_sessions()

jobs = []

for sub in subjects:
    for ses in sessions:
        # Find all anatomical brain masks (space-T1w and desc-brain_mask)
        anat_masks = layout.get(
            subject=sub, session=ses, datatype="anat",
            desc="brain",suffix="mask", extension="nii.gz"
                )[0]
        anat_mask_path = anat_mask.path

        # Find corresponding functional bold reference images
        hmc_boldrefs = layout.get(
                    subject=sub, session=ses, datatype="func",
                    desc="hmc",suffix="boldref", extension="nii.gz"
                )

        for bold_ref in hmc_boldrefs:
            bold_ref_path = bold_ref.path

            xfm_file = layout.get(
                subject=sub, session=ses, datatype="func",
                task=bold_ref.task, run=bold_ref.run,
                desc="hmc",suffix="xfm", extension="txt"
                    )
            xfm_txt_path=xfm_file[0].path                    
            # Construct the output path
            output_desc_mask = bold_ref_path.replace(
                "desc-hmc_boldref.nii.gz", "desc-brain_mask.nii.gz"
            )
            
            if all(map(os.path.exists, [anat_mask_path, bold_ref_path, xfm_txt_path])):
                jobs.append((anat_mask_path, bold_ref_path, xfm_txt_path, output_desc_mask))
            else:
                print(f"Skipping: Missing files for {sub} {ses}")

# Use multiprocessing for faster processing
if __name__ == "__main__":
    with Pool(processes=8) as pool:  # Adjust number of processes
        pool.starmap(process_brain_mask, jobs)

    print("Processing completed.")
